# Phe-EWAS replication

In [1]:
# LOAD LIBRARIES
library(devtools)
library(roxygen2)
library(clarite)
library(moments)
library(ggplot2)
library(gridExtra)

Loading required package: usethis



In [2]:
# LOAD DATASETS
load("../Data/nh_99-06.Rdata")

# Load variable descriptions for annotation when saving files
descriptions_info <- unique(VarDescription[,colnames(VarDescription) %in% c("tab_desc","module","var","var_desc")])

In [3]:
# Change ID column name
colnames(MainTable)[1] <- "ID" 

In [38]:
# Add survey year and combine LBDHDD and LBDHDL columns (they are the same variable)
survey_year <- colfilter(MainTable, c("ID"))
MainTable[MainTable$SDDSRVYR==3 | MainTable$SDDSRVYR==4, "LBDHDL"] = MainTable[MainTable$SDDSRVYR==3 | MainTable$SDDSRVYR==4, "LBDHDD"] 

[1] "Running..."
[1] "Finished in 0.000145 secs"


In [42]:
MainTable$SMQ077[MainTable$SMQ077==7] <- NA
MainTable$DBD100[MainTable$DBD100==9] <- NA

In [49]:
# GETTING SURVEY INFORMATION
#Get survey info
survey_design_discovery <- read.csv(paste("/home/tomas/Documents/RepPheEWAS/Data/weights_discovery.txt", sep="/"), sep="\t")
colnames(survey_design_discovery)[1] <- "ID"
survey_design_discovery <- colfilter(survey_design_discovery, c("SDDSRVYR"), exclude=TRUE)

#Get survey info
survey_design_replication <- read.csv(paste("/home/tomas/Documents/RepPheEWAS/Data/weights_replication.txt", sep="/"), sep="\t")
colnames(survey_design_replication)[1] <- "ID"
survey_design_replication <- colfilter(survey_design_replication, c("SDDSRVYR"), exclude=TRUE)

# Divide replication weights by 2 to get 4 year weights
survey_design_replication[,-(1:4)] <- survey_design_replication[,-(1:4)] / 2

[1] "Running..."
[1] "Finished in 9.8e-05 secs"
[1] "Running..."
[1] "Finished in 9.2e-05 secs"


In [47]:
# Get weight mapping data
var_weights <- read.csv(paste("/home/tomas/Documents/RepPheEWAS/Data/VarWeights.csv", sep="/"), fileEncoding="UTF-8-BOM", stringsAsFactors = FALSE)
var_weights <- var_weights[complete.cases(var_weights), ]
weights_discovery <- setNames(as.list(var_weights$discovery), as.list(var_weights$variable_name))
weights_replication <- setNames(as.list(var_weights$replication), as.list(var_weights$variable_name))


# Remove unnecessary variables

How many variables to we have at the start?

In [55]:
length(MainTable)

[1] 1191

In [56]:
# Remove physical fitness vars and unknown pesticide variable
no_pf_vars_URXP08 <- c("CVDVOMAX","CVDESVO2","CVDS1HR","CVDS1SY","CVDS1DI","CVDS2HR","CVDS2SY", "CVDS2DI","CVDR1HR","CVDR1SY","CVDR1DI","CVDR2HR","CVDR2SY","CVDR2DI","physical_activity", "URXP08") 
MainTable <- colfilter(d = MainTable, cols = c(no_pf_vars_URXP08), exclude = TRUE)

# Remove other variables (inconclusive meaning)
others <- c("house_type","hepa","hepb", "house_age", "current_past_smoking","house_age", "DUQ130", "DMDMARTL", "income")
MainTable <- colfilter(MainTable, cols=others, exclude = TRUE)

#Set covariates and phenotypes 
covariates <- c("female", "SES_LEVEL", "SDDSRVYR", "RIDAGEYR", "other_eth", "other_hispanic", "black", "mexican", "BMXBMI")
phenotype <- c("LBXSCRINV", "URXUCR",	"LBXSCR", "LBXSATSI",	"LBXSAL",	"URXUMASI",	"URXUMA",	"LBXSAPSI",	"LBXSASSI",	"LBXSC3SI",	"LBXSBU",	"LBXBAP",	"LBXCPSI",	"LBXCRP",	"LBXSCLSI",	"LBXSCH",	"LBDHDL",	"LBDLDL",	"LBXFER",	"LBXSGTSI",	"LBXSGB",	"LBXGLU",	"LBXGH",	"LBXHCY",	"LBXSIR",	"LBXSLDSI",	"LBXMMA",	"LBXSOSSI",	"LBXSPH",	"LBXSKSI",	"LBXEPP",	"LBXSNASI",	"LBXTIB",	"LBXSTB",	"LBXSCA",	"LBXSTP",	"LBDPCT",	"LBXSTR",	"LBXSUA",	"LBDBANO",	"LBXBAPCT",	"LBDEONO",	"LBXEOPCT",	"LBXHCT",	"LBXHGB",	"LBDLYMNO",	"LBXMCHSI",	"LBXLYPCT",	"LBXMCVSI",	"LBXMPSI",	"LBDMONO",	"LBXMOPCT",	"LBXPLTSI",	"LBXRBCSI",	"LBXRDW",	"LBDNENO",	"LBXNEPCT", "LBXIRN")
phenotype_rep <- c("LBXSCRINV", "URXUCR",	"LBXSCR", "LBXSATSI",	"LBXSAL",	"URXUMASI",	"URXUMA",	"LBXSAPSI",	"LBXSASSI",	"LBXSC3SI",	"LBXSBU",	"LBXBAP",	"LBXCPSI",	"LBXCRP",	"LBXSCLSI",	"LBXSCH",	"LBDHDL",	"LBDLDL",	"LBXFER",	"LBXSGTSI",	"LBXSGB",	"LBXGLU",	"LBXGH",	"LBXHCY",	"LBXSIR",	"LBXSLDSI",	"LBXMMA",	"LBXSOSSI",	"LBXSPH", "LBXSKSI",	"LBXEPP",	"LBXSNASI",	"LBXTIB",	"LBXSTB",	"LBXSCA",	"LBXSTP",	"LBDPCT",	"LBXSTR",	"LBXSUA",	"LBDBANO",	"LBXBAPCT",	"LBDEONO",	"LBXEOPCT",	"LBXHCT",	"LBXHGB",	"LBDLYMNO",	"LBXMCHSI",	"LBXLYPCT",	"LBXMCVSI",	"LBXMPSI",	"LBDMONO",	"LBXMOPCT",	"LBXPLTSI",	"LBXRBCSI",	"LBXRDW",	"LBDNENO",	"LBXNEPCT", "LBXIRN")

#Remove variables that do not have 4 year weights 
remove_variables_lbxtc <- c("LBXTC")
MainTable <- colfilter(MainTable, remove_variables_lbxtc, exclude=TRUE)

[1] "Running..."
[1] "Finished in 5e-04 secs"
[1] "Running..."
[1] "Finished in 0.000286 secs"
[1] "Running..."
[1] "Finished in 0.000231 secs"


How many do we end up with?

In [57]:
length(MainTable)

[1] 1166

# Spliting variables by type

In [58]:
bin_phe <- get_binary(MainTable)
cat_phe <- get_categorical(MainTable)
cont_phe <- get_continuous(MainTable)
check_phe <- get_check(MainTable)

[1] "Running..."
[1] "Finished in 0.739235 secs"
[1] "Running..."
[1] "Finished in 1.319574 secs"
[1] "Running..."
[1] "Warning: area may contain non-numeric values."
[1] "Finished in 0.637832 secs"
[1] "Running..."
[1] "Finished in 1.277391 secs"


How many by type?

In [66]:
sprintf("There are %i binary, %i categorical, %i continuos, and %i variables to check", length(bin_phe), length(cat_phe), length(cont_phe), length(check_phe))

[1] "There are 416 binary, 57 categorical, 607 continuos, and 39 variables to check"

In [89]:
# Some columns need to be moved from categorical to continuous
cat_to_cont_phe <- c("L_ASPARTIC_ACID_mg", "L_GLUTAMINE_gm", "DBQ095", "DRDDRSTZ", "BETA_CAROTENE_mcg", "CALCIUM_PPM", "GLYCINE_gm", "LBXPFDO", "LBDBANO", "LBXTO3", "DRD350AQ",	"DRD350BQ",	"DRD350CQ",	"DRD350DQ",	"DRD350EQ",	"DRD350FQ",	"DRD350GQ",	"DRD350IQ",	"DRD350JQ",	"DRD370AQ",	"DRD370CQ",	"DRD370DQ",	"DRD370EQ",	"DRD370FQ",	"DRD370GQ",	"DRD370HQ",	"DRD370IQ",	"DRD370KQ",	"DRD370NQ",	"DRD370RQ",	"DRD370SQ")
cont_phe <- merge_data(cont_phe, colfilter(cat_phe, cols = cat_to_cont_phe, exclude = FALSE))
cat_phe <- colfilter(cat_phe, cols = cat_to_cont_phe, exclude = TRUE)

[1] "Running..."
[1] "Running..."
[1] "Finished in 0.000777 secs"
[1] "Finished in 0.251947 secs"
[1] "Running..."
[1] "Finished in 0.000151 secs"


In [90]:
sprintf("There are %i binary, %i categorical, %i continuos, and %i variables to check", length(bin_phe), length(cat_phe), length(cont_phe), length(check_phe))

[1] "There are 416 binary, 52 categorical, 612 continuos, and 39 variables to check"

In [92]:
# All "check" columns are continuous
print(paste("Checked: ", paste(names(check_phe), collapse=", "), sep=" "))
cont_phe <- merge_data(cont_phe, check_phe, union = TRUE)

[1] "Checked:  ID, LBXPFDO, LBDBANO, LBXTO3, DRD350AQ, DRD350BQ, DRD350CQ, DRD350DQ, DRD350EQ, DRD350FQ, DRD350GQ, DRD350IQ, DRD350JQ, DRD370AQ, DRD370CQ, DRD370DQ, DRD370EQ, DRD370FQ, DRD370GQ, DRD370HQ, DRD370IQ, DRD370KQ, DRD370NQ, DRD370RQ, DRD370SQ, ALCOHOL_PERCENT, L_ASPARTIC_ACID_mg, L_GLUTAMINE_gm, LBXV2T, LBXV4T, LBXVDM, LBXPFBS, LBDSY3, LBXTO5, URXMMI, DUQ280, DUQ360, OMEGA_9_FATTY_ACIDS_mg, URXEMM"
[1] "Running..."
[1] "Finished in 0.298981 secs"


In [94]:
sprintf("There are %i binary, %i categorical, and %i continuos variables", length(bin_phe), length(cat_phe), length(cont_phe), length(check_phe))

[1] "There are 416 binary, 52 categorical, and 650 continuos variables"

In [95]:
# Get covariate types
covariates_cat_phe = intersect(names(cat_phe), covariates)
covariates_bin_phe = intersect(names(bin_phe), covariates)
covariates_catbin_phe <- union(covariates_cat_phe, covariates_bin_phe)
covariates_cont_phe = intersect(names(cont_phe), covariates)

In [96]:
covariates_catbin_phe

[1] "SES_LEVEL"      "SDDSRVYR"       "female"         "black"         
[5] "mexican"        "other_hispanic" "other_eth"

In [97]:
covariates_cont_phe

[1] "RIDAGEYR" "BMXBMI"

In [98]:
statins <- c("ATORVASTATIN_CALCIUM", "SIMVASTATIN", "PRAVASTATIN_SODIUM", "FLUVASTATIN_SODIUM")
phestatin <- colfilter(bin_phe, cols = statins)
phestatin$total <- rowSums(phestatin[2:5], na.rm = TRUE)

[1] "Running..."
[1] "Finished in 0.000645 secs"


In [114]:
sprintf("There are %i participants out of %i on statins", sum(phestatin$total!=0), dim(MainTable)[1])

[1] "There are 1059 participants out of 41474 on statins"